In [1]:
#importing the Packages
import pandas as pd
import nltk
from nltk import *
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from random import shuffle


from nltk.classify import ClassifierI
from statistics import mode
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression

#Type of stemmer used and allowed word types
ps = PorterStemmer()
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words("english"))
stop_words.update(('@','.', '..', '...', '....', ':', ':-)', ':)', '!', '!!', '!!!', '?', '??',','))
allowed_word_types = ["J","R","V","N"]    #j is adjective, r is adverb, v is verb and n is noun


In [2]:
# Preprocessing the data
document = []
untokenised_doc = []
labels = []
for i, chunk in enumerate(pd.read_csv("Twitter.csv", chunksize = 1000,header = None)):
    a = chunk.iloc[:,0]
    for j in range(len(a)):
        words = []
        stemmed_words = []
        lemm_words = []
        df1 = str(chunk.iloc[j,5])
        words = word_tokenize(df1)
        for w in words:
            stemmed_words.append(ps.stem(w))
        for w in stemmed_words:
            lemm_words.append(lemmatizer.lemmatize(w))
        untokenised_doc.append(" ".join(lemm_words))
        labels.append(chunk.iloc[j,0])

In [3]:
# Stratified K-fold
kf = StratifiedKFold(n_splits=10)

for train_number, test_number in kf.split(untokenised_doc,labels):
    X_train = [untokenised_doc[i] for i in train_number]
    X_test = [untokenised_doc[i] for i in test_number]
    y_train = [labels[i] for i in train_number]
    y_test = [labels[i] for i in test_number]
    vectorizer = TfidfVectorizer(min_df=5, max_df = 0.8, sublinear_tf=True, use_idf=True,stop_words = stop_words)
    train_doc_tf_idf = vectorizer.fit_transform(X_train) 
    test_doc_tf_idf = vectorizer.transform(X_test)

In [4]:
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression

model1 = LinearSVC()
model2 = MultinomialNB()
model3 = BernoulliNB()
model4 = SGDClassifier()
model5 = LogisticRegression()

In [5]:
model1.fit(train_doc_tf_idf, y_train)
model2.fit(train_doc_tf_idf, y_train)
model3.fit(train_doc_tf_idf, y_train)
model4.fit(train_doc_tf_idf, y_train)
model5.fit(train_doc_tf_idf, y_train)

/Users/georgeboben/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [6]:
ypred1 = model1.predict(test_doc_tf_idf)
ypred2 = model2.predict(test_doc_tf_idf)
ypred3 = model3.predict(test_doc_tf_idf)
ypred4 = model4.predict(test_doc_tf_idf)
ypred5 = model5.predict(test_doc_tf_idf)

In [7]:
result1 = accuracy_score(y_test, ypred1, normalize = True)
result2 = accuracy_score(y_test, ypred2, normalize = True)
result3 = accuracy_score(y_test, ypred3, normalize = True)
result4 = accuracy_score(y_test, ypred4, normalize = True)
result5 = accuracy_score(y_test, ypred5, normalize = True)

In [8]:
print(result1)
print(result2)
print(result3)
print(result4)
print(result5)

0.6965448172258387
0.7050575863795694
0.7025538307461192
0.699549323985979
0.700550826239359
